# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Sep 2 2022 12:55PM,245639,12,HH6709,Hush Hush,Released
1,Sep 2 2022 12:55PM,245639,12,HH6710,Hush Hush,Released
2,Sep 2 2022 12:55PM,245639,12,HH6718,Hush Hush,Released
3,Sep 2 2022 12:51PM,245636,19,ADV80005424,"AdvaGen Pharma, Ltd",Executing
4,Sep 2 2022 12:51PM,245636,19,ADV80005425,"AdvaGen Pharma, Ltd",Executing
5,Sep 2 2022 12:51PM,245636,19,ADV80005426,"AdvaGen Pharma, Ltd",Executing
6,Sep 2 2022 12:51PM,245636,19,ADV80005427,"AdvaGen Pharma, Ltd",Executing
7,Sep 2 2022 12:51PM,245636,19,ADV80005428,"AdvaGen Pharma, Ltd",Released
8,Sep 2 2022 12:51PM,245636,19,ADV80005429,"AdvaGen Pharma, Ltd",Executing
9,Sep 2 2022 12:21PM,245633,10,0085958865,ISDIN Corporation,Executing


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
38,245633,Executing,21
39,245633,Released,30
40,245636,Executing,5
41,245636,Released,1
42,245639,Released,3


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
245631,NaN,NaN,1.0
245632,NaN,NaN,1.0
245633,NaN,21.0,30.0
245636,NaN,5.0,1.0
245639,NaN,NaN,3.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
245455,5.0,1.0,0.0
245482,7.0,12.0,12.0
245562,8.0,5.0,1.0
245563,0.0,0.0,1.0
245564,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
245455,5,1,0
245482,7,12,12
245562,8,5,1
245563,0,0,1
245564,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,245455,5,1,0
1,245482,7,12,12
2,245562,8,5,1
3,245563,0,0,1
4,245564,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,245455,5,1,
1,245482,7,12,12
2,245562,8,5,1
3,245563,,,1
4,245564,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Sep 2 2022 12:55PM,245639,12,Hush Hush
3,Sep 2 2022 12:51PM,245636,19,"AdvaGen Pharma, Ltd"
9,Sep 2 2022 12:21PM,245633,10,ISDIN Corporation
60,Sep 2 2022 11:47AM,245632,12,Hush Hush
61,Sep 2 2022 11:43AM,245630,20,Alumier Labs Inc.
67,Sep 2 2022 11:43AM,245631,10,"Methapharm, Inc."
68,Sep 2 2022 11:20AM,245626,16,Sartorius Stedim Filters Inc.
69,Sep 2 2022 11:20AM,245625,16,Sartorius Stedim Filters Inc.
70,Sep 2 2022 11:17AM,245624,18,TASA USA Inc.
71,Sep 2 2022 11:14AM,245622,20,"ACI Healthcare USA, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Sep 2 2022 12:55PM,245639,12,Hush Hush,,,3
1,Sep 2 2022 12:51PM,245636,19,"AdvaGen Pharma, Ltd",,5,1
2,Sep 2 2022 12:21PM,245633,10,ISDIN Corporation,,21,30
3,Sep 2 2022 11:47AM,245632,12,Hush Hush,,,1
4,Sep 2 2022 11:43AM,245630,20,Alumier Labs Inc.,,,6
5,Sep 2 2022 11:43AM,245631,10,"Methapharm, Inc.",,,1
6,Sep 2 2022 11:20AM,245626,16,Sartorius Stedim Filters Inc.,,,1
7,Sep 2 2022 11:20AM,245625,16,Sartorius Stedim Filters Inc.,,,1
8,Sep 2 2022 11:17AM,245624,18,TASA USA Inc.,,,1
9,Sep 2 2022 11:14AM,245622,20,"ACI Healthcare USA, Inc.",,1,10


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 2 2022 12:55PM,245639,12,Hush Hush,3,,
1,Sep 2 2022 12:51PM,245636,19,"AdvaGen Pharma, Ltd",1,5,
2,Sep 2 2022 12:21PM,245633,10,ISDIN Corporation,30,21,
3,Sep 2 2022 11:47AM,245632,12,Hush Hush,1,,
4,Sep 2 2022 11:43AM,245630,20,Alumier Labs Inc.,6,,
5,Sep 2 2022 11:43AM,245631,10,"Methapharm, Inc.",1,,
6,Sep 2 2022 11:20AM,245626,16,Sartorius Stedim Filters Inc.,1,,
7,Sep 2 2022 11:20AM,245625,16,Sartorius Stedim Filters Inc.,1,,
8,Sep 2 2022 11:17AM,245624,18,TASA USA Inc.,1,,
9,Sep 2 2022 11:14AM,245622,20,"ACI Healthcare USA, Inc.",10,1,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 2 2022 12:55PM,245639,12,Hush Hush,3,,
1,Sep 2 2022 12:51PM,245636,19,"AdvaGen Pharma, Ltd",1,5,
2,Sep 2 2022 12:21PM,245633,10,ISDIN Corporation,30,21,
3,Sep 2 2022 11:47AM,245632,12,Hush Hush,1,,
4,Sep 2 2022 11:43AM,245630,20,Alumier Labs Inc.,6,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 2 2022 12:55PM,245639,12,Hush Hush,3.0,NaN,NaN
1,Sep 2 2022 12:51PM,245636,19,"AdvaGen Pharma, Ltd",1.0,5.0,NaN
2,Sep 2 2022 12:21PM,245633,10,ISDIN Corporation,30.0,21.0,NaN
3,Sep 2 2022 11:47AM,245632,12,Hush Hush,1.0,NaN,NaN
4,Sep 2 2022 11:43AM,245630,20,Alumier Labs Inc.,6.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 2 2022 12:55PM,245639,12,Hush Hush,3.0,0.0,0.0
1,Sep 2 2022 12:51PM,245636,19,"AdvaGen Pharma, Ltd",1.0,5.0,0.0
2,Sep 2 2022 12:21PM,245633,10,ISDIN Corporation,30.0,21.0,0.0
3,Sep 2 2022 11:47AM,245632,12,Hush Hush,1.0,0.0,0.0
4,Sep 2 2022 11:43AM,245630,20,Alumier Labs Inc.,6.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,1964885,102.0,28.0,0.0
12,491271,4.0,0.0,0.0
15,736671,18.0,24.0,7.0
16,2210389,8.0,1.0,0.0
17,245585,1.0,0.0,0.0
18,245624,1.0,0.0,0.0
19,245636,1.0,5.0,0.0
20,1718977,20.0,7.0,13.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,1964885,102.0,28.0,0.0
1,12,491271,4.0,0.0,0.0
2,15,736671,18.0,24.0,7.0
3,16,2210389,8.0,1.0,0.0
4,17,245585,1.0,0.0,0.0
5,18,245624,1.0,0.0,0.0
6,19,245636,1.0,5.0,0.0
7,20,1718977,20.0,7.0,13.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,102.0,28.0,0.0
1,12,4.0,0.0,0.0
2,15,18.0,24.0,7.0
3,16,8.0,1.0,0.0
4,17,1.0,0.0,0.0
5,18,1.0,0.0,0.0
6,19,1.0,5.0,0.0
7,20,20.0,7.0,13.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,102.0
1,12,Released,4.0
2,15,Released,18.0
3,16,Released,8.0
4,17,Released,1.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,17,18,19,20
Status,,,,,,,,
Completed,0.0,0.0,7.0,0.0,0.0,0.0,0.0,13.0
Executing,28.0,0.0,24.0,1.0,0.0,0.0,5.0,7.0
Released,102.0,4.0,18.0,8.0,1.0,1.0,1.0,20.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,17,18,19,20
0,Completed,0.0,0.0,7.0,0.0,0.0,0.0,0.0,13.0
1,Executing,28.0,0.0,24.0,1.0,0.0,0.0,5.0,7.0
2,Released,102.0,4.0,18.0,8.0,1.0,1.0,1.0,20.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,17,18,19,20
0,Completed,0.0,0.0,7.0,0.0,0.0,0.0,0.0,13.0
1,Executing,28.0,0.0,24.0,1.0,0.0,0.0,5.0,7.0
2,Released,102.0,4.0,18.0,8.0,1.0,1.0,1.0,20.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()